# 📓 The GenAI Revolution Cookbook

**Title:** How to Build a Model Context Protocol (MCP) Server in Python

**Description:** Learn how to build an MCP server in Python to standardize and reuse AI tools, resources, and prompts across applications. This hands-on guide walks you through server setup, client testing, and GPT-4 chatbot integration for production-ready systems.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



The Model Context Protocol defines how clients discover and call tools, read resources, and render prompts from a server. Any MCP-capable client can connect and use your definitions without one-off glue code. For a comprehensive introduction to MCP's core concepts and why standardization matters, see our [Model Context Protocol (MCP) Explained [2025 Guide for Builders]](/article/model-context-protocol-mcp-explained-2025-guide-for-builders).

This guide walks you through building a minimal Python MCP server that exposes two arithmetic tools (`add` and `subtract`), a static documentation resource, and a parameterized prompt template. You'll validate the server with a Python client, then map its tool schemas to OpenAI's function-calling format and complete a single conversational turn with GPT-4.

By the end, you'll have:
- A local MCP server exposing tools, resources, and prompts over stdio
- A Python client that lists and calls these capabilities
- A working integration that routes OpenAI tool calls to your MCP server and returns a final answer

---

## Why Use MCP for This Problem

Without MCP, every agent or chatbot you build requires custom glue code to wire tools, prompts, and data sources. You duplicate integration logic, drift schemas across projects, and manually sync updates.

MCP solves this by standardizing discovery and invocation. Write your tools once as an MCP server; any MCP-capable client—Claude Desktop, custom agents, or OpenAI-backed chatbots—can discover and call them without per-client adapters. You centralize tool definitions, versioning, and access control in one place.

Compared to alternatives:
- **Ad-hoc tool wiring**: Requires custom code per client; no schema discovery.
- **OpenAPI specs**: REST-only; no native support for prompts or resources.
- **LangChain Tools**: Framework-specific; not interoperable outside LangChain.
- **gRPC**: Requires code generation and lacks built-in prompt/resource abstractions.

MCP provides a lightweight, transport-agnostic protocol with first-class support for tools, resources, and prompts—ideal for reusable, discoverable AI capabilities.

---

## Core Concepts for This Use Case

**Server**: An MCP server exposes capabilities (tools, resources, prompts) to clients. You define it using the `Server` class from the `mcp.server` module.

**Tools**: Functions the client can invoke. Each tool has a name, description, and JSON Schema for input validation. Decorate functions with `@server.tool()` to register them.

**Resources**: Static or dynamic content identified by URI (e.g., `docs://calc/quickstart`). Use `@server.resource()` to serve documentation, policies, or datasets.

**Prompts**: Parameterized templates that clients can render with arguments. Use `@server.prompt()` to define reusable instruction patterns.

**Stdio transport**: MCP servers and clients communicate over standard input/output. The `stdio_server()` context manager handles message framing; `StdioClient` launches the server as a subprocess and connects to it.

**ClientSession**: Manages the lifecycle of a client connection. Use `session.initialize()` to handshake, then call `list_tools()`, `call_tool()`, `read_resource()`, and `render_prompt()` to interact with the server.

---

## Setup

Run this cell to install the MCP SDK and OpenAI client library:

In [ ]:
!pip install -q "mcp[cli]" "openai"

Set your OpenAI API key (required for the final integration step):

In [ ]:
import os
from getpass import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass("Enter your OpenAI API key: ")

Create the server module file in the notebook environment:

In [ ]:
from pathlib import Path

server_code = '''
import asyncio
import logging
from typing import Any, Dict, List

from mcp.server import Server
from mcp.server.stdio import stdio_server
from mcp.types import TextContent

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("mcp_calc_server")

server = Server("calc-mcp")

@server.tool(
    "add",
    description="Add two numbers",
    input_schema={
        "type": "object",
        "properties": {
            "a": {"type": "number"},
            "b": {"type": "number"}
        },
        "required": ["a", "b"],
    },
)
async def add_tool(args: Dict[str, Any]) -> List[TextContent]:
    """
    Adds two numbers provided in the arguments.

    Args:
        args (Dict[str, Any]): Dictionary with keys 'a' and 'b' (numbers).

    Returns:
        List[TextContent]: List containing the sum as a TextContent object.

    Raises:
        ValueError: If 'a' or 'b' is missing or not a number.
    """
    try:
        a = float(args["a"])
        b = float(args["b"])
        result = a + b
        logger.info(f"add_tool called with a={a}, b={b}, result={result}")
        return [TextContent(type="text", text=str(result))]
    except (KeyError, ValueError, TypeError) as e:
        logger.error(f"Invalid input for add_tool: {args} ({e})")
        raise ValueError("Both 'a' and 'b' must be valid numbers.")

@server.tool(
    "subtract",
    description="Subtract b from a",
    input_schema={
        "type": "object",
        "properties": {
            "a": {"type": "number"},
            "b": {"type": "number"}
        },
        "required": ["a", "b"],
    },
)
async def subtract_tool(args: Dict[str, Any]) -> List[TextContent]:
    """
    Subtracts 'b' from 'a' provided in the arguments.

    Args:
        args (Dict[str, Any]): Dictionary with keys 'a' and 'b' (numbers).

    Returns:
        List[TextContent]: List containing the difference as a TextContent object.

    Raises:
        ValueError: If 'a' or 'b' is missing or not a number.
    """
    try:
        a = float(args["a"])
        b = float(args["b"])
        result = a - b
        logger.info(f"subtract_tool called with a={a}, b={b}, result={result}")
        return [TextContent(type="text", text=str(result))]
    except (KeyError, ValueError, TypeError) as e:
        logger.error(f"Invalid input for subtract_tool: {args} ({e})")
        raise ValueError("Both 'a' and 'b' must be valid numbers.")

DOCS_URI = "docs://calc/quickstart"

@server.resource(DOCS_URI, description="Calculator server quickstart")
async def read_docs() -> List[TextContent]:
    """
    Returns documentation for the calculator MCP server.

    Returns:
        List[TextContent]: List containing the documentation as a TextContent object.
    """
    content = (
        "Calculator MCP Server\\n"
        "- Tools: add(a,b), subtract(a,b)\\n"
        "- All numeric inputs are coerced to float.\\n"
        "- Results are returned as text."
    )
    return [TextContent(type="text", text=content)]

PROMPT_NAME = "calc_instructions"

@server.prompt(
    PROMPT_NAME,
    description="Explain a calculation before returning the numeric result.",
    arguments_schema={
        "type": "object",
        "properties": {
            "task": {"type": "string", "description": "Natural language math task"}
        },
        "required": ["task"],
    },
)
async def render_calc_prompt(args: Dict[str, Any]) -> List[TextContent]:
    """
    Renders a prompt explaining a calculation task.

    Args:
        args (Dict[str, Any]): Dictionary with key 'task' (string).

    Returns:
        List[TextContent]: List containing the rendered prompt as a TextContent object.
    """
    task = str(args["task"])
    template = (
        "You are a careful math assistant.\\n"
        "Task: {task}\\n"
        "Explain your steps briefly, then provide the final numeric answer."
    )
    return [TextContent(type="text", text=template.format(task=task))]

async def main():
    """
    Entry point for running the MCP server over stdio.
    """
    async with stdio_server() as (read, write):
        await server.run(read, write)

if __name__ == "__main__":
    asyncio.run(main())
'''

Path("mcp_calc_server.py").write_text(server_code)
print("Server module created: mcp_calc_server.py")

---

## Build the MCP Server

The server code above defines:
- **Two tools** (`add` and `subtract`) that accept numeric arguments and return text results
- **One resource** (`docs://calc/quickstart`) that serves static documentation
- **One prompt** (`calc_instructions`) that renders a parameterized instruction template

Each tool validates inputs, coerces them to floats, and logs the operation. The server runs over stdio, reading JSON-RPC messages from stdin and writing responses to stdout.

---

## Validate with a Python Client

This cell launches the server as a subprocess, connects a client, and tests tool discovery, resource reading, and prompt rendering:

In [ ]:
import asyncio
import sys
from mcp.client.stdio import StdioClient
from mcp.client.session import ClientSession

async def test_server():
    """
    Launches the MCP server as a subprocess and tests tool/resource/prompt discovery and usage.
    """
    cmd = [sys.executable, "mcp_calc_server.py"]
    async with StdioClient(cmd) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # List available tools
            tools = await session.list_tools()
            print("Tools:", [t.name for t in tools])

            # List available resources
            resources = await session.list_resources()
            print("Resources:", [r.uri for r in resources])

            # List available prompts
            prompts = await session.list_prompts()
            print("Prompts:", [p.name for p in prompts])

            # Call 'add' tool and print result
            result = await session.call_tool("add", {"a": 3, "b": 5})
            print("add(3,5) =>", "".join([c.text for c in result if hasattr(c, "text")]))

            # Read documentation resource and print content
            doc_content = await session.read_resource("docs://calc/quickstart")
            print("Docs:", "".join([c.text for c in doc_content if hasattr(c, "text")]))

            # Render a prompt and print the result
            rendered = await session.render_prompt("calc_instructions", {"task": "Add 12 and 30"})
            print("Rendered Prompt:", "".join([c.text for c in rendered if hasattr(c, "text")]))

            await session.shutdown()

await test_server()

You should see:
- Tool names: `['add', 'subtract']`
- Resource URI: `['docs://calc/quickstart']`
- Prompt name: `['calc_instructions']`
- `add(3,5) => 8.0`
- Documentation text
- Rendered prompt with the task filled in

---

## Map MCP Tools to OpenAI Function Calling

OpenAI's function-calling API expects tool definitions in a specific format. This function converts MCP tool schemas to OpenAI-compatible definitions:

In [ ]:
def mcp_tools_to_openai(tools):
    """
    Converts a list of MCP tool objects to OpenAI function-calling tool definitions.

    Args:
        tools (list): List of MCP tool objects.

    Returns:
        list: List of OpenAI-compatible tool definitions (dicts).
    """
    openai_tools = []
    for t in tools:
        # Access input_schema attribute (SDK uses snake_case)
        input_schema = getattr(t, "input_schema", None) or {"type": "object", "properties": {}}
        openai_tools.append({
            "type": "function",
            "function": {
                "name": t.name,
                "description": t.description or "",
                "parameters": input_schema
            }
        })
    return openai_tools

---

## Prepare the OpenAI Client and Tool Catalog

This cell initializes the OpenAI client and fetches tool definitions from the MCP server, converting them to OpenAI format:

In [ ]:
import sys
from openai import OpenAI

client = OpenAI()

async def prepare_tools():
    """
    Fetches tool definitions from the MCP server and converts them to OpenAI format.

    Returns:
        list: List of OpenAI-compatible tool definitions.
    """
    cmd = [sys.executable, "mcp_calc_server.py"]
    async with StdioClient(cmd) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()
            tools = await session.list_tools()
            await session.shutdown()
            return mcp_tools_to_openai(tools)

openai_tools = await prepare_tools()
print("OpenAI tool catalog prepared:", [t["function"]["name"] for t in openai_tools])

---

## Make an Initial Chat Completion

This cell sends a user question to GPT-4 with the tool catalog. The model decides whether to call a function:

In [ ]:
async def first_turn(openai_tools):
    """
    Sends an initial chat completion request to OpenAI with tool catalog.

    Args:
        openai_tools (list): List of OpenAI-compatible tool definitions.

    Returns:
        OpenAI response object.
    """
    resp = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "You can call functions to perform precise math."},
            {"role": "user", "content": "What is 12.5 + 30.2?"}
        ],
        tools=openai_tools,
        tool_choice="auto",
        temperature=0  # Deterministic for arithmetic
    )
    return resp

initial_resp = await first_turn(openai_tools)
print("Model response:", initial_resp.choices[0].message)

If the model decides to call a function, `tool_calls` will be populated in the response.

---

## Route Tool Calls to the MCP Server

This cell detects tool calls in the OpenAI response, routes them to the MCP server, and formats the results for OpenAI:

In [ ]:
import json

async def route_tool_calls(resp, session):
    """
    Routes tool calls from OpenAI response to the MCP server and formats results.

    Args:
        resp: OpenAI response object.
        session: Active MCP ClientSession.

    Returns:
        list: List of tool result messages formatted for OpenAI.
    """
    tool_calls = resp.choices[0].message.tool_calls or []
    tool_results_msgs = []
    for tc in tool_calls:
        name = tc.function.name
        try:
            args = json.loads(tc.function.arguments or "{}")
        except json.JSONDecodeError as e:
            print(f"Failed to parse arguments for {name}: {e}")
            args = {}
        # Call the corresponding MCP tool
        result_parts = await session.call_tool(name, args)
        result_text = "".join([c.text for c in result_parts if hasattr(c, "text")])
        tool_results_msgs.append({
            "role": "tool",
            "tool_call_id": tc.id,
            "content": result_text
        })
    return tool_results_msgs

---

## Finalize the Answer

This cell sends the original assistant message, tool call metadata, and tool results back to OpenAI to get a final user-facing answer:

In [ ]:
def finalize_answer(initial_resp, tool_msgs):
    """
    Sends the original assistant message, tool call metadata, and tool results to OpenAI for a final answer.

    Args:
        initial_resp: Initial OpenAI response object.
        tool_msgs (list): List of tool result messages.

    Returns:
        OpenAI response object with the final answer.
    """
    messages = [
        {"role": "system", "content": "You can call functions to perform precise math."},
        {"role": "user", "content": "What is 12.5 + 30.2?"},
        initial_resp.choices[0].message,
    ]
    messages.extend(tool_msgs)

    final = client.chat.completions.create(
        model="gpt-4o",
        messages=messages,
        temperature=0
    )
    return final

---

## Run the Full Conversation

This cell orchestrates the entire flow: prepare tools, ask a question, route tool calls, and print the final answer:

In [ ]:
async def chat():
    """
    Orchestrates a full conversation: fetches tools, asks a question, routes tool calls, and prints the final answer.
    """
    openai_tools = await prepare_tools()
    cmd = [sys.executable, "mcp_calc_server.py"]
    async with StdioClient(cmd) as (read, write):
        async with ClientSession(read, write) as session:
            await session.initialize()

            # First turn: ask the model a question
            initial = await first_turn(openai_tools)
            # Route any tool calls to the MCP server
            tool_msgs = await route_tool_calls(initial, session)
            # Get the final answer from OpenAI
            final = finalize_answer(initial, tool_msgs)

            print("Final answer:", final.choices[0].message.content)
            await session.shutdown()

await chat()

You should see a final answer like: `"The result of 12.5 + 30.2 is 42.7."`

---

## Run and Evaluate

Run the full conversation cell above to confirm the entire pipeline works end-to-end. You should observe:
- The MCP server starts as a subprocess
- The client fetches tool definitions and converts them to OpenAI format
- GPT-4 receives the user question and decides to call the `add` tool
- The client routes the tool call to the MCP server and receives `42.7`
- GPT-4 receives the tool result and generates a natural language answer

If you encounter errors:
- **Missing API key**: Ensure `OPENAI_API_KEY` is set in the environment.
- **Import errors**: Verify `mcp` and `openai` are installed.
- **Attribute errors**: Check that `input_schema` (not `inputSchema`) is used in `mcp_tools_to_openai`.
- **JSON decode errors**: Inspect `tc.function.arguments` for malformed JSON; add logging to `route_tool_calls`.

For production use, add:
- **Logging**: Use `logging.info()` to trace tool calls and results.
- **Schema validation**: Validate tool arguments against the input schema before calling.
- **Error handling**: Catch and surface exceptions from tool execution to the client.

---

## Conclusion

You've built a minimal MCP server exposing arithmetic tools, a documentation resource, and a parameterized prompt. You validated it with a Python client and integrated it with OpenAI's function-calling API to complete a conversational turn.

Next steps:
- Replace the calculator tools with real API calls (e.g., weather, database queries, or web search)
- Add evaluation and observability with Langfuse to track tool call accuracy and latency
- Containerize the server with Docker for deployment as a standalone service
- Explore MCP's sampling and roots features for advanced agent workflows

Your server is now a reusable, discoverable interface to capabilities—ready to power any MCP-capable client.